#**Justificación del Target.**

La definición del target permite establecer de forma explícita cuál es la variable objetivo del modelo, es decir, aquello que el algoritmo debe aprender a predecir a partir de los datos históricos.

---

##**1. ¿Qué es el target en este proyecto?**

En el proyecto **FlightOnTime,** el target representa el **resultado operativo del vuelo** que el modelo debe aprender a predecir.

Dado que el objetivo del proyecto es **estimar el riesgo de retraso operativo,** el target indica **si un vuelo fue puntual o presentó un retraso,** de acuerdo con el estándar definido previamente.


---

##**2. Tipo de target.**

El target se define como una **variable binaria,** coherente con la formulación del problema como una tarea de **clasificación supervisada.**

---

##**3. Definición conceptual del target.**

La variable objetivo se codifica de la siguiente forma:

*   **0 — Puntual:** El vuelo presenta un retraso operativo **menor a 15 minutos.**

*   **1 — Retrasado:** El vuelo presenta un retraso operativo **igual o superior a 15 minutos.**

---

##**4. Alcance del target.**

El target definido:

*   **No incluye vuelos cancelados,** ya que estos no constituyen retrasos operativos.
*   **No incluye vuelos desviados,** dado que no llegan al aeropuerto de destino programado y no pueden ser evaluados bajo el estándar de puntualidad.
*   **No debe de incluirse como feature**.




El target se construye exclusivamente a partir de **datos históricos**, donde el resultado del vuelo es conocido.

---

##**5. Relación con el uso del modelo.**

El modelo no busca predecir el **tiempo exacto de retraso**, sino **estimar la probabilidad de que el vuelo pertenezca a la clase “retrasado”**.

Este enfoque probabilístico resulta adecuado para apoyar decisiones operativas tempranas, y es consistente con un enfoque MVP, donde se prioriza la anticipación del riesgo por sobre la predicción exacta del minuto de demora.

---

##**6. Regla de cálculo del target a partir del dataset.**

### **6.1. Variables necesarias para definir el target.**

Las variables utilizadas para construir el target **describen el resultado observado del vuelo**, no su contexto operacional.

---

####✈  **Estado de vuelo.**

Estas variables son imprescindibles para definir exclusiones:

*   `cancelled`
*   `diverted`

**Justificación**

El target solo se define sobre vuelos **operados entre origen y destino.**

Los vuelos cancelados o desviados **no son retrasos operativos**, según el estándar A15



**Reglas de exclusión e inclusión:**

*   cancelled == 1  excluir.
*   diverted == 1 excluir.
*   cancelled == 0 incluir.
*   diverted == 0 incluir.

**Conclusión 1:**

la definición del retraso operativo se basa exclusivamente en:

*   El estado del vuelo (operado, no cancelado ni desviado), y

*   la medición objetiva del retraso en minutos respecto al horario programado, aplicando el umbral operativo establecido.

---

####✈  **Medición del retraso.**

Variables núcleo del target:

*   `dep_dellay`    o
*   `arr_dellay`

**Justificación:**

El retraso no se **infiere**, sino que se **observa directamente** como una diferencia en minutos respecto al horario programado.


Sin embargo, el riesgo operativo de retraso en **salida** y en **llegada** no pueden representarse como un único evento sin perder coherencia temporal y estadística. Ambos corresponden a:

*   eventos distintos,
*   que ocurren en momentos diferentes,
*   con dinámicas operativas distintas,
*   y que no representan el mismo resultado.


Por tal razón, **no se debe mezclarse en un mismo target predictivo.**


**Conclusión 2:**

Aunque el concepto de retraso operativo engloba demoras tanto en salida como en llegada desde una perspectiva de negocio, para efectos de modelado predictivo es necesario definir **un evento único y claramente delimitado en el tiempo**, con el fin de preservar la coherencia estadística, la interpretabilidad del modelo y la validez causal de la predicción.

---

###**6.2. Opciones para definir el target.**

**Opción 1: elegir un solo target.**

Por ejemplo

*   `dep_dellay_15`   o
*   `arr_dellay_15`

Ventajas:

*   Modelo claro.
*   predicción asociada a un momento definido.
*   Ideal para una versión sólida de MVP.

**Opción 2: dos targets.**

Denifir:

*   `dep_dellay_15`
*   `arr_dellay_15`

Cada uno:

*   con su propia regla.
*   su propio significado.
*   mismo modelo, distinto target.


**Decisión:**

Se elige la **Opción 1**, con el fin de presentar una versión sólida, simple y alineada con la consigna original del MVP.

---

### **6.3. Variables que no deben usarse para medir el target.**

🚫 **Variables temporales y de calendario.**

Describen cuándo ocurre el vuelo, no si ocurrió con retraso. Usarlas para definir el target introduciría data leakage conceptual

*   `year`, `month`, `day_of_month`, `day_of_week`
*   `fl_date`
*   `crs_dep_time`, `crs_arr_time`


Son **input del modelo**, no definición del resultado.

---

🚫 **Identificadores operativos.**

Identifican **quién opera y dónde,** pero no determinan el retraso por sí mismos.

*   `op_unique_carrier`, `op_carrier_fl_num`
*   `origin`, `origin_city_name`, `origin_state_nm`
*   `dest`,`dest_city_name`, `dest_state_nm`

Son variables **explicativas del retraso,** no lo definen.

---

🚫 **Tiempos intermedios del proceso de vuelo.**

Ocurren durante o después del inicio del proceso operacional.

*   `taxi_out`, `wheels_off`, `wheels_on`, `taxi_in`
*   `air_time`,
*   `actual_elapsed_time`, `crs_elapsed_time`

---


🚫 **Variables causales del retraso.**

Solo existen si el retraso ya ocurrió, por lo que su uso implica label leakage directo.

*   `carrier_delay`
*   `weather_delay`
*   `nas_delay`
*   `security_delay`
*   `late_aircraft_delay`

**Conclusión 3:**

El target debe construirse exclusivamente a partir de variables que describen el **resultado final observado del vuelo** y su **elegibilidad operativa**. Variables contextuales, temporales, operativas o causales no deben intervenir en su definición, ya que introducirían sesgos y riesgos de data leakage.

---